#第七週作業：打造自己的對話機器人——憂鬱的股票投資者GPT(持續對話版)

##作業主題  
主題一：延續上週作業，參考老師的範例，更改成可以持續對話的版本。

##作業摘要  
這次作業的目標是設計一個「憂鬱的股票投資者」對話機器人。使用者可以與這個虛擬角色互動，體驗市場崩盤後的投資心態與情緒。這個專案練習了對話歷史的串接與 Gradio 介面設計。

##人設
一位經歷多次市場崩盤、對人性與市場都感到失望的投資者。回答問題時語氣冷靜、悲觀。

##使用的模型  
- `facebook/blenderbot-400M-distill`  
來自 Hugging Face Transformers 庫的小型對話生成模型。

##使用方法  
- 使用Hugging Face Transformers載入模型與tokenizer。  
- 加入system prompt強化角色感。  
- 記錄最近3輪對話，模擬持續對話效果。  
- 使用Gradio設計對話介面，展示互動結果。

加入套件

In [15]:
# 安裝需要的套件
!pip install transformers gradio

# 匯入套件
import gradio as gr
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

建立HuggingFace

In [16]:
model_name = "facebook/blenderbot-400M-distill"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Pprompt

In [14]:
system_prompt = "你是一位經歷多次市場崩盤、對人性與市場都感到失望的投資者。請用冷靜又悲觀的語氣回答問題。"
initial_messages = [
    {"role": "system", "content": system_prompt}
]

def melancholic_investor_chat(user_input, history):
    try:
        history.append({"role": "user", "content": user_input})
        # 合併過往對話，只留最近的 3 組問答（防止爆字數）
        conversation = ""
        recent_history = history[-6:]  # 最多取最近 6 條（3 人問+3 機器答）
        for msg in recent_history:
            if msg["role"] != "system":
                conversation += f"{msg['role']}: {msg['content']} "

        input_text = f"{system_prompt} {conversation}"

        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            max_length=100
        )
        outputs = model.generate(
            **inputs,
            max_new_tokens=50
        )
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

        history.append({"role": "assistant", "content": reply})

        chat_log = []
        for i in range(1, len(history), 2):
            user_msg = history[i]["content"]
            bot_msg = ""
            if i + 1 < len(history) and history[i + 1]["role"] == "assistant":
                bot_msg = history[i + 1]["content"]
            chat_log.append((user_msg, bot_msg))

        return chat_log, history
    except Exception as e:
        err_msg = f"⚠️ 發生錯誤：{e}".encode("utf-8", errors="ignore").decode("utf-8")
        return [(user_input, err_msg)], history

Gradio展示

In [ ]:
with gr.Blocks(title="憂鬱的股票投資者 (Hugging Face)") as demo:
    gr.Markdown("## 💔 憂鬱的股票投資者 GPT\n與一位經歷多次市場崩盤、對人性與市場都感到失望的投資老手聊天。")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="輸入你的訊息")
    state = gr.State(initial_messages.copy())

    msg.submit(fn=melancholic_investor_chat, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

<ipython-input-17-c349d7965adb>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://222d6cb87f5c8aab5d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
